# BAN CARBON Revenue Forecast

This notebook implements the revenue forecasting methodology described in `/home/fdvom/ban-carbon-hq/research/revenue-forecast/context/forecasting-methodology.md`.

## Overview

The forecast computes M-month grassroots donation revenue based on:
- **Channels**: Ways to reach potential donors
- **Segments**: Donor groups with shared characteristics
- **Funnel parameters**: Reachable audience, lead rates, conversion rates
- **Donor behavior**: Gift amounts and attrition rates

## Revenue Model

```
revenue = Σ_c Σ_s [ n_cs * lead_c * conv * Σ_{t=m_c}^M gift_s * (1 - attr)^{t - m_c} ]
```

In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime

## 1. Load Data

In [7]:
# Define paths
data_dir = Path("../../../data/raw/revenue-forecasts")

# Load core tables
channels = pd.read_csv(data_dir / "channels.csv")
segments = pd.read_csv(data_dir / "segments.csv")

# Load lead rates and gift amounts
leads_raw = pd.read_csv(data_dir / "lead-rates/combined.csv")
gifts_raw = pd.read_csv(data_dir / "gift-amounts/combined.csv")

# Average lead rates across LLMs
leads = leads_raw[['id_cha']].copy()
leads['lead'] = leads_raw[['lead_chatgpt', 'lead_claude', 'lead_gemini']].mean(axis=1)

# Average gift amounts across LLMs
gifts = gifts_raw[['id_seg']].copy()
gifts['gift'] = gifts_raw[['gift_chatgpt', 'gift_claude', 'gift_gemini']].mean(axis=1)

# Load audience data (ChatGPT only - most conservative)
audience = pd.read_csv(data_dir / "audience-size/chatgpt.csv")

# Load channel sequencing from all three sources
seq_chatgpt = pd.read_csv(data_dir / "channel-sequence/chatgpt.csv")
seq_claude = pd.read_csv(data_dir / "channel-sequence/claude.csv")
seq_gemini = pd.read_csv(data_dir / "channel-sequence/gemini.csv")

# Merge and average sequences
seq_chatgpt = seq_chatgpt.rename(columns={'month': 'month_chatgpt'})
seq_claude = seq_claude.rename(columns={'month': 'month_claude'})
seq_gemini = seq_gemini.rename(columns={'month': 'month_gemini'})

channel_seq = seq_chatgpt[['id_cha', 'month_chatgpt']].merge(
    seq_claude[['id_cha', 'month_claude']], on='id_cha', how='outer'
).merge(
    seq_gemini[['id_cha', 'month_gemini']], on='id_cha', how='outer'
)
channel_seq['month'] = channel_seq[['month_chatgpt', 'month_claude', 'month_gemini']].mean(axis=1).round().astype(int)
channel_seq = channel_seq[['id_cha', 'month']]

# Load structural parameters
struct_params = pd.read_csv(data_dir / "structural-parameters.csv")
conv = struct_params['conv'].iloc[0]
attr = struct_params['attr'].iloc[0]
M = 12

print(f"✓ Loaded {len(channels)} channels, {len(segments)} segments")
print(f"✓ Parameters: conv={conv:.1%}, attr={attr:.1%}, M={M} months")

✓ Loaded 77 channels, 14 segments
✓ Parameters: conv=12.0%, attr=2.2%, M=12 months


## 2. Calculate Revenue

In [25]:
# Create base table
base = audience.merge(leads[['id_cha', 'lead']], on='id_cha', how='left')
base = base.merge(gifts[['id_seg', 'gift']], on='id_seg', how='left')
base = base.merge(channel_seq[['id_cha', 'month']], on='id_cha', how='left')
base = base.rename(columns={'month': 'm_c', 'lead': 'lead_c', 'gift': 'gift_s', 'n': 'n_cs'})

# Compute number of donors from each channel-segment that are active each month
base["starting_donors"] = base["n_cs"] * base["lead_c"] * conv
cols = ["id_cha", "id_seg", "starting_donors", "gift_s", "lead_c", "m_c", "n_cs"]
base = base[cols]
months = np.tile(np.arange(1, M+1), len(base))
base = pd.DataFrame(np.repeat(base.values, M, axis = 0), columns = cols)
base["month"] = months
base["months_active"] = base["month"] - base["m_c"]
base["donors"] = base["starting_donors"] * (1 - attr)**base["months_active"]
base.loc[base["months_active"] < 0, "donors"] = 0

# Compute revenue
base["revenue"] = base["donors"] * base["gift_s"]

# Filter out channels not reached within M
base = base[base['m_c'] <= M]

# Print total revenue
total_revenue = base['revenue'].sum()
print(f"\n✓ Total M-mo Revenue: ${total_revenue:,.2f}")
print(f"✓ Channel-segment pairs in forecast: {len(base[["id_cha", "id_seg"]].drop_duplicates())}")


✓ Total M-mo Revenue: $105,179.97
✓ Channel-segment pairs in forecast: 22


## 3. Fetch tables for Revenue Detail tab

In [38]:
# Get segment-level data
ids = ["id_seg", "month"]
df = base[ids + ["donors", "revenue"]].groupby(ids).sum().reset_index()
df = df.merge(gifts, on = "id_seg", how = "left")

# Get segment-level lead and conversion rates
df_seg = base[["id_seg", "id_cha", "n_cs", "lead_c"]].drop_duplicates()
df_seg["numerator"] = df_seg["lead_c"] * df_seg["n_cs"]
df_seg = df_seg.groupby("id_seg").sum().reset_index()
df_seg["lead_s"] = df_seg["numerator"] / df_seg["n_cs"]
df_seg = df_seg[["id_seg", "lead_s"]]
df_seg["conv"] = conv

# Get donor and revenue monthly matrix
df_rev = df.pivot(index = "id_seg", columns = "month", values = "revenue")
df_don = df.pivot(index = "id_seg", columns = "month", values = "donors")

month_cols = list(range(1, M+1))

# Sort segments by earliest activation month
segment_order = base.groupby('id_seg')['m_c'].min().reset_index()
segment_order = segment_order.sort_values('m_c')
segment_order["order_seg"] = np.arange(len(segment_order))

# Enhance df_rev: add segment name, conv, lead_s, and description
df_rev = df_rev.merge(segments[['id_seg', 'name_seg', 'description_seg']], on='id_seg')
df_rev = df_rev.merge(df_seg[['id_seg', 'conv', 'lead_s']], on='id_seg')
df_rev = df_rev.merge(segment_order, on = "id_seg", how = "left")
df_rev = df_rev.sort_values("order_seg")
df_rev = df_rev[['name_seg', 'description_seg', 'lead_s', 'conv'] + month_cols]

# Enhance df_don: add segment name, conv, lead_s, and description
df_don = df_don.merge(segments[['id_seg', 'name_seg', 'description_seg']], on='id_seg')
df_don = df_don.merge(df_seg[['id_seg', 'conv', 'lead_s']], on='id_seg')
df_don = df_don.merge(segment_order, on = "id_seg", how = "left")
df_don = df_don.sort_values("order_seg")
df_don = df_don[['name_seg', 'description_seg', 'lead_s', 'conv'] + month_cols]

# Round month columns: df_rev to nearest integer
df_rev[month_cols] = df_rev[month_cols].round(0).astype(int)
## For df_don: round to 1 if value is between 0 and 1, otherwise round normally
for col in month_cols:
    df_don[col] = df_don[col].apply(lambda x: 1 if 0 < x < 1 else round(x))
df_don[month_cols] = df_don[month_cols].astype(int)

# Rename columns for display
rename_dict = {
    'name_seg': 'Segment',
    'description_seg': 'Segment Description',
    'lead_s': 'Lead Rate',
    'conv': 'Conversion Rate'
}
for i in range(1, M+1):
    rename_dict[i] = f'Month {i}'

df_rev = df_rev.rename(columns=rename_dict)
df_don = df_don.rename(columns=rename_dict)

# Get month column names after renaming
month_col_names = [f'Month {i}' for i in range(1, M+1)]

# Add TOTAL column to df_rev
df_rev['TOTAL'] = df_rev[month_col_names].sum(axis=1)

# Add totals row to df_rev
totals_row_rev = pd.DataFrame([{
    'Segment': 'TOTAL',
    'Segment Description': '',
    'Lead Rate': np.nan,
    'Conversion Rate': np.nan,
    **{col: df_rev[col].sum() for col in month_col_names},
    'TOTAL': df_rev['TOTAL'].sum()
}])
df_rev = pd.concat([df_rev, totals_row_rev], ignore_index=True)

# Add totals row to df_don
totals_row_don = pd.DataFrame([{
    'Segment': 'TOTAL',
    'Segment Description': '',
    'Lead Rate': np.nan,
    'Conversion Rate': np.nan,
    **{col: df_don[col].sum() for col in month_col_names}
}])
df_don = pd.concat([df_don, totals_row_don], ignore_index=True)

## 3. Calculate Monthly Breakdown

In [2]:
# Calculate monthly revenue directly from base
monthly_revenue = base.groupby('month')[['donors', 'revenue']].sum().reset_index()
monthly_revenue.columns = ['month', 'total_donors', 'revenue']
monthly_revenue = monthly_revenue.sort_values('month')
monthly_revenue['cumulative_revenue'] = monthly_revenue['revenue'].cumsum()

print("\n✓ Monthly breakdown calculated")
monthly_revenue

NameError: name 'base' is not defined

## 4. Calculate Expenses

In [ ]:
#TODO


✓ Total Expenses: $121,339.83
✓ Net Position: $-16,159.85


,month,founder_comp,contract_support,software_infra,accounting_legal,transaction_fees,fiscal_sponsor_fees,marketing,total_expenses,cumulative_expenses
0,1,5000,3000,500,800,0.000000,0.000000,2000,11300.000000,11300.000000
1,2,5000,3000,500,800,100.672000,335.573333,2000,11736.245333,23036.245333
2,3,5000,2000,500,800,118.873216,396.244053,1500,10315.117269,33351.362603
3,4,5000,2000,500,800,276.867605,922.892017,1500,10999.759623,44351.122225
4,5,5000,1500,500,800,270.776518,902.588393,1000,9973.364911,54324.487136
5,6,5000,1500,500,800,318.477035,1061.590115,1000,10180.067150,64504.554286
6,7,5000,1000,500,800,398.779340,1329.264466,800,9828.043806,74332.598092
7,8,5000,1000,500,800,443.930882,1479.769608,800,10023.700490,84356.298582
8,9,5000,500,500,800,453.364403,1511.214676,600,9364.579079,93720.877661
9,10,5000,500,500,800,452.542386,1508.474620,600,9361.017006,103081.894667


## 5. Generate Excel Budget

Create comprehensive 5-tab Excel workbook for fiscal sponsor review.

In [42]:
# Create Excel workbook
wb = Workbook()
ws = wb.active
ws.title = "Revenue Detail"

# Write "Donors" header
ws['A1'] = "Donors"

# Write df_don table starting at A2
start_row = 2
for r_idx, row in enumerate(dataframe_to_rows(df_don, index=False, header=True), start=start_row):
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Calculate starting row for Revenue section
revenue_start_row = start_row + len(df_don) + 1 + 2  # +1 for header, +2 to skip two rows

# Write "Revenue" header
ws.cell(row=revenue_start_row, column=1, value="Revenue (USD)")

# Write df_rev table
start_row_rev = revenue_start_row + 1
for r_idx, row in enumerate(dataframe_to_rows(df_rev, index=False, header=True), start=start_row_rev):
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Create Methodology sheet
ws_methodology = wb.create_sheet("Methodology")

# Read methodology file
methodology_path = Path("/home/fdvom/ban-carbon-hq/research/budget/context/forecasting-methodology.md")
with open(methodology_path, 'r') as f:
    methodology_content = f.read()

# Split content by lines and write each line to a separate row in column A
lines = methodology_content.split('\n')
for row_idx, line in enumerate(lines, start=1):
    ws_methodology.cell(row=row_idx, column=1, value=line)

# Adjust column width for better readability
ws_methodology.column_dimensions['A'].width = 100

# Create Data Inputs sheet
ws_data = wb.create_sheet("Data Inputs")

# Helper function to write a table with title
def write_table_with_title(ws, title, df, start_row):
    # Write title
    ws.cell(row=start_row, column=1, value=title)
    # Write table starting one row below title
    table_start = start_row + 1
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), start=table_start):
        for c_idx, value in enumerate(row, start=1):
            ws.cell(row=r_idx, column=c_idx, value=value)
    # Return next available row (current position + df length + header + 2 blank rows)
    return table_start + len(df) + 1 + 2

# Write all data tables
current_row = 1
current_row = write_table_with_title(ws_data, "Channels", channels, current_row)
current_row = write_table_with_title(ws_data, "Segments", segments, current_row)
current_row = write_table_with_title(ws_data, "Lead Rates", leads, current_row)
current_row = write_table_with_title(ws_data, "Gift Amounts", gifts, current_row)
current_row = write_table_with_title(ws_data, "Channel Sequence", channel_seq, current_row)
current_row = write_table_with_title(ws_data, "Structural Parameters", struct_params, current_row)
current_row = write_table_with_title(ws_data, "Audience Size", audience, current_row)

# Save workbook
output_path = Path("../reports/revenue-forecast-budget.xlsx")
wb.save(output_path)